In [59]:
from ipynb.fs.full.data_preprocessing import send_freq_data, send_time_data

freq_witness, strain_data = send_freq_data()
time_witness, strain_data = send_time_data()

print(freq_witness.shape)
print(freq_witness)
#print(strain_data)

(32, 32768, 21)
[[[ 3.45925393e-04 -3.47274076e-03 -5.37202061e-09 ...  1.43218656e-07
    3.05353183e-07 -1.73395369e-02]
  [-1.85789104e-04  1.86412979e-03  2.89339286e-09 ... -7.70794912e-08
   -1.63774715e-07  2.44587902e-02]
  [ 2.23084207e-04 -2.24002148e-03 -3.46064133e-09 ...  9.22875074e-08
    1.97023908e-07 -4.45013773e-03]
  ...
  [ 4.69461447e-05 -1.31375040e-04 -3.40881057e-09 ...  1.60314301e-07
    4.70688271e-08  1.69079576e-03]
  [ 4.70481391e-05 -1.33088732e-04 -3.40860762e-09 ...  1.60324390e-07
    4.71100492e-08  4.89403168e-03]
  [ 4.71515705e-05 -1.34792994e-04 -3.40838890e-09 ...  1.60334196e-07
    4.71511754e-08  8.08258727e-03]]

 [[ 7.89340760e-04  2.28750007e-03 -6.07063866e-09 ...  1.69187103e-07
   -3.56135388e-07  1.31263509e-01]
  [-4.24342172e-04 -1.23217807e-03  3.26501004e-09 ... -9.08889959e-08
    1.91010585e-07 -8.12526345e-02]
  [ 5.08856552e-04  1.47352973e-03 -3.91282029e-09 ...  1.09098877e-07
   -2.29791240e-07  7.76870251e-02]
  ...
  [-9.8

In [7]:
#Dimension 1 = batch
#Dimension 2 = datapoint
#Dimension 3 = witness channel

print(freq_witness.shape)
print(freq_witness[0].shape)
print(freq_witness[0])
freq_witness[0].max()
freq_witness[0].min()

(32, 32768, 21)
(32768, 21)
[[ 3.4592539e-04 -3.4727408e-03 -5.3720206e-09 ...  1.4321866e-07
   3.0535318e-07 -1.7339537e-02]
 [-1.8578910e-04  1.8641298e-03  2.8933929e-09 ... -7.7079491e-08
  -1.6377471e-07  2.4458790e-02]
 [ 2.2308421e-04 -2.2400215e-03 -3.4606413e-09 ...  9.2287507e-08
   1.9702391e-07 -4.4501377e-03]
 ...
 [ 4.6946145e-05 -1.3137504e-04 -3.4088106e-09 ...  1.6031430e-07
   4.7068827e-08  1.6907958e-03]
 [ 4.7048139e-05 -1.3308873e-04 -3.4086076e-09 ...  1.6032439e-07
   4.7110049e-08  4.8940317e-03]
 [ 4.7151570e-05 -1.3479299e-04 -3.4083889e-09 ...  1.6033420e-07
   4.7151175e-08  8.0825873e-03]]


-0.034083776

In [55]:
#split the time data
from sklearn.model_selection import train_test_split
train_time, test_time = train_test_split(
    time_witness[0], test_size = 0.1, random_state = 42)

print(train_time.shape, test_time.shape)

(29491, 21) (3277, 21)


# Attempt 3 (WIP)

In [56]:
#Attempting to run the full 3d dataset didn't work
#Let's try taking a single batch and running it through

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

#Check input shape, given dataset is sampled at 4096Hz
model = keras.models.Sequential(name="attempt_3")
#Convolution Layers
model.add(layers.Conv1D(filters=21, kernel_size=5, strides=1, padding="same", activation="tanh",\
                        input_shape=(train_time.shape[0], train_time.shape[1])))
model.add(layers.BatchNormalization())
model.add(layers.Conv1D(filters=8, kernel_size=5, strides=2, padding="same", activation="tanh"))
model.add(layers.BatchNormalization())
model.add(layers.Conv1D(filters=16, kernel_size=5, strides=2, padding="same", activation="tanh"))
model.add(layers.BatchNormalization())
model.add(layers.Conv1D(filters=32, kernel_size=5, strides=2, padding="same", activation="tanh"))
model.add(layers.BatchNormalization())
model.add(layers.Conv1D(filters=64, kernel_size=5, strides=2, padding="same", activation="tanh"))
model.add(layers.BatchNormalization())

#Deconvolution Layers
model.add(layers.Conv1DTranspose(filters=32, kernel_size=5, strides=2, padding="same", activation="tanh"))
model.add(layers.BatchNormalization())
model.add(layers.Conv1DTranspose(filters=16, kernel_size=5, strides=2, padding="same", activation="tanh"))
model.add(layers.BatchNormalization())
model.add(layers.Conv1DTranspose(filters=8, kernel_size=5, strides=2, padding="same", activation="tanh"))
model.add(layers.BatchNormalization())
model.add(layers.Conv1DTranspose(filters=21, kernel_size=5, strides=2, padding="same", activation="tanh"))
model.add(layers.BatchNormalization())
model.add(layers.Conv1D(filters=1, kernel_size=5, padding="same", name = "output"))

model.build((None, train_time.shape[0], train_time.shape[1]))
model.summary()

Model: "attempt_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_45 (Conv1D)          (None, 29491, 21)         2226      
                                                                 
 batch_normalization_81 (Bat  (None, 29491, 21)        84        
 chNormalization)                                                
                                                                 
 conv1d_46 (Conv1D)          (None, 14746, 8)          848       
                                                                 
 batch_normalization_82 (Bat  (None, 14746, 8)         32        
 chNormalization)                                                
                                                                 
 conv1d_47 (Conv1D)          (None, 7373, 16)          656       
                                                                 
 batch_normalization_83 (Bat  (None, 7373, 16)         64

In [62]:
#define mean relative error loss function
import scipy.integrate as integrate
def mean_relative_error(freq_witness, )
f2 = freq_witness[0].max()
f1 = freq_witness[0].min()
recip = 1 / (f2 - f1)
integrand = np.sqrt(residual/strain_data)
integrate.quad(integrand, f1, f2)


14.684299502637792


In [53]:
#Toy code, need to double check lr_schedule parameters with literature
import numpy as np

#train_time = train_time.transpose()
#test_time = test_time.transpose()

lr_schedule = keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=1e-3, decay_steps=100, decay_rate=0.8)

model.compile(loss="binary_crossentropy", optimizer=keras.optimizers.Adam(learning_rate=lr_schedule), metrics=["accuracy"])

results = model.fit(
    train_time,
    test_time,
    batch_size=32,
    epochs=10,
    verbose=1,
    validation_split=0.1,
    callbacks=[
        keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=5, verbose=1, min_lr=1e-5),
        keras.callbacks.EarlyStopping(patience=10, verbose=1),
    ],
)

Epoch 1/10


ValueError: in user code:

    File "/home/mvigil/.local/lib/python3.9/site-packages/keras/engine/training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "/home/mvigil/.local/lib/python3.9/site-packages/keras/engine/training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/mvigil/.local/lib/python3.9/site-packages/keras/engine/training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "/home/mvigil/.local/lib/python3.9/site-packages/keras/engine/training.py", line 1023, in train_step
        y_pred = self(x, training=True)
    File "/home/mvigil/.local/lib/python3.9/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/home/mvigil/.local/lib/python3.9/site-packages/keras/engine/input_spec.py", line 250, in assert_input_compatibility
        raise ValueEr

# Attempt 2 (scrapped)

In [35]:
#Implement neural network.

#TOY MODEL COPY-PASTED FROM HW3
#MODEL FOR PART B, MODIFIED TO REPLICATE
#FIGURE 7 IN THE SOURCE WORK
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

#Check input shape, given dataset is sampled at 4096Hz
model = keras.models.Sequential(name="attempt_2")
#Convolution Layers
model.add(layers.Conv1D(filters=21, kernel_size=5, strides=1, padding="same", activation="tanh",\
                        input_shape=(train_time.shape[1], train_time.shape[2])))
model.add(layers.BatchNormalization())
model.add(layers.Conv1D(filters=8, kernel_size=5, strides=2, padding="same", activation="tanh"))
model.add(layers.BatchNormalization())
model.add(layers.Conv1D(filters=16, kernel_size=5, strides=2, padding="same", activation="tanh"))
model.add(layers.BatchNormalization())
model.add(layers.Conv1D(filters=32, kernel_size=5, strides=2, padding="same", activation="tanh"))
model.add(layers.BatchNormalization())
model.add(layers.Conv1D(filters=64, kernel_size=5, strides=2, padding="same", activation="tanh"))
model.add(layers.BatchNormalization())

#Deconvolution Layers
model.add(layers.Conv1DTranspose(filters=32, kernel_size=5, strides=2, padding="same", activation="tanh"))
model.add(layers.BatchNormalization())
model.add(layers.Conv1DTranspose(filters=16, kernel_size=5, strides=2, padding="same", activation="tanh"))
model.add(layers.BatchNormalization())
model.add(layers.Conv1DTranspose(filters=8, kernel_size=5, strides=2, padding="same", activation="tanh"))
model.add(layers.BatchNormalization())
model.add(layers.Conv1DTranspose(filters=21, kernel_size=5, strides=2, padding="same", activation="tanh"))
model.add(layers.BatchNormalization())
model.add(layers.Conv1D(filters=1, kernel_size=5, padding="same", name = "output"))

model.build((None, train_time.shape[1], train_time.shape[2]))
model.summary()

Model: "attempt_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_15 (Conv1D)          (None, 32768, 21)         2226      
                                                                 
 batch_normalization_27 (Bat  (None, 32768, 21)        84        
 chNormalization)                                                
                                                                 
 conv1d_16 (Conv1D)          (None, 16384, 8)          848       
                                                                 
 batch_normalization_28 (Bat  (None, 16384, 8)         32        
 chNormalization)                                                
                                                                 
 conv1d_17 (Conv1D)          (None, 8192, 16)          656       
                                                                 
 batch_normalization_29 (Bat  (None, 8192, 16)         64

In [36]:
#Toy code, need to double check lr_schedule parameters with literature
import numpy as np
lr_schedule = keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=1e-3, decay_steps=100, decay_rate=0.8)

model.compile(loss="binary_crossentropy", optimizer=keras.optimizers.Adam(learning_rate=lr_schedule), metrics=["accuracy"])

results = model.fit(
    train_time,
    test_time,
    batch_size=32,
    epochs=10,
    verbose=1,
    validation_split=0.1,
    callbacks=[
        keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=5, verbose=1, min_lr=1e-5),
        keras.callbacks.EarlyStopping(patience=10, verbose=1),
    ],
)

ValueError: Data cardinality is ambiguous:
  x sizes: 25
  y sizes: 4
Make sure all arrays contain the same number of samples.

# Attempt 1 (scrapped)

In [ ]:
#Implement neural network.

#TOY MODEL COPY-PASTED FROM HW3
#MODEL FOR PART B, MODIFIED TO REPLICATE
#FIGURE 7 IN THE SOURCE WORK
from tensorflow.keras import layers

#Check input shape, given dataset is sampled at 4096Hz
model = keras.models.Sequential(name="attempt_1")
#Convolution Layers
model.add(layers.Conv1D(filters=21, kernel_size=5, strides=1, padding="same", activation="tanh",\
                        input_shape=(X_batch.shape[1], X_batch.shape[2])))
model.add(layers.BatchNormalization())
model.add(layers.Conv1D(filters=8, kernel_size=5, strides=2, padding="same", activation="tanh"))
model.add(layers.BatchNormalization())
model.add(layers.Conv1D(filters=16, kernel_size=5, strides=2, padding="same", activation="tanh"))
model.add(layers.BatchNormalization())
model.add(layers.Conv1D(filters=32, kernel_size=5, strides=2, padding="same", activation="tanh"))
model.add(layers.BatchNormalization())
model.add(layers.Conv1D(filters=64, kernel_size=5, strides=2, padding="same", activation="tanh"))
model.add(layers.BatchNormalization())

#Deconvolution Layers
model.add(layers.Conv1DTranspose(filters=32, kernel_size=5, strides=2, padding="same", activation="tanh"))
model.add(layers.BatchNormalization())
model.add(layers.Conv1DTranspose(filters=16, kernel_size=5, strides=2, padding="same", activation="tanh"))
model.add(layers.BatchNormalization())
model.add(layers.Conv1DTranspose(filters=8, kernel_size=5, strides=2, padding="same", activation="tanh"))
model.add(layers.BatchNormalization())
model.add(layers.Conv1DTranspose(filters=21, kernel_size=5, strides=2, padding="same", activation="tanh"))
model.add(layers.BatchNormalization())
model.add(layers.Conv1D(filters=1, kernel_size=5, padding="same", name = "output"))

model.build((None, X_batch.shape[1], X_batch.shape[0]))
model.summary()